In [44]:
#from google.colab import drive
#drive.mount('/content/drive')

In [45]:
import pandas as pd

filepath_dict = {'yelp':   'sentiment-labelled-sentences/yelp_labelled.txt',
                 'amazon': 'sentiment-labelled-sentences/amazon_cells_labelled.txt',
                 'imdb':   'sentiment-labelled-sentences/imdb_labelled.txt'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)

df = pd.concat(df_list)
print(df.head())

                                            sentence  label source
0                           Wow... Loved this place.      1   yelp
1                                 Crust is not good.      0   yelp
2          Not tasty and the texture was just nasty.      0   yelp
3  Stopped by during the late May bank holiday of...      1   yelp
4  The selection on the menu was great and so wer...      1   yelp


In [46]:
from sklearn.model_selection import train_test_split

sentences = df['sentence'].values
y = df['label'].values
# First split 20% of the data into testing and validation
sentences_train, sentences_test_val, y_train, y_test_val = train_test_split(sentences, y, test_size=0.2, random_state=1000)
# Then split 50% of the test+val data as validation
sentences_test, sentences_val, y_test, y_val = train_test_split(sentences_test_val, y_test_val, test_size=0.5, random_state=1000)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(2198,)
(275,)
(275,)


## Transform our sentences into one-hot representations using CountVectorizer

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)
X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)
X_val = vectorizer.transform(sentences_val)

## Building a logistic regression benchmark

In [48]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.8472727272727273


## Pytorch basics

In [49]:
input_dim = X_train.shape[1]
print (input_dim)

4642


In [50]:
import torch
import torch.nn as nn

class feedforward_nn(nn.Module):
    def __init__(self, input_dim):
        super(feedforward_nn, self).__init__()
        self.fc1 = nn.Linear(input_dim, 10)
        self.fc2 = nn.Linear(10, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = feedforward_nn(input_dim)
# Print model summary
print (model)

feedforward_nn(
  (fc1): Linear(in_features=4642, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [51]:
import numpy as np
import random

def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

In [52]:
set_seed(3)

import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_val_tensor = torch.tensor(X_val.toarray(), dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Create a PyTorch dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Define batch size
batch_size = 20
epochs = 10

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Training loop
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    for inputs, labels in train_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Squeeze the output tensor to match the shape of labels
        outputs = torch.squeeze(outputs, dim=1)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

    # Validation
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            outputs = torch.squeeze(outputs, dim=1)  # Squeeze the output tensor
            val_loss += criterion(outputs, labels).item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = correct / total

    # Print the loss and accuracy
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.3f}, Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}')

Epoch 1/10, Loss: 0.613, Val Loss: 0.660, Val Accuracy: 0.695
Epoch 2/10, Loss: 0.563, Val Loss: 0.587, Val Accuracy: 0.793
Epoch 3/10, Loss: 0.389, Val Loss: 0.514, Val Accuracy: 0.807
Epoch 4/10, Loss: 0.451, Val Loss: 0.465, Val Accuracy: 0.807
Epoch 5/10, Loss: 0.223, Val Loss: 0.435, Val Accuracy: 0.825
Epoch 6/10, Loss: 0.101, Val Loss: 0.423, Val Accuracy: 0.825
Epoch 7/10, Loss: 0.092, Val Loss: 0.417, Val Accuracy: 0.822
Epoch 8/10, Loss: 0.086, Val Loss: 0.412, Val Accuracy: 0.815
Epoch 9/10, Loss: 0.043, Val Loss: 0.414, Val Accuracy: 0.822
Epoch 10/10, Loss: 0.057, Val Loss: 0.418, Val Accuracy: 0.818


In [53]:
set_seed(3)

X_test_tensor = torch.tensor(X_test.toarray(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create a PyTorch dataset for test data
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Define batch size for test data
test_batch_size = 20

# Create DataLoader for test data
test_loader = DataLoader(test_dataset, batch_size=test_batch_size)

model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        outputs = torch.squeeze(outputs, dim=1)  # Squeeze the output tensor
        test_loss += criterion(outputs, labels).item()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = correct / total

# Print the testing accuracy
print("Testing Accuracy: {:.4f}".format(test_accuracy))

Testing Accuracy: 0.8364


## Using word embeddings

In [54]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hz6974/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [56]:
from nltk import word_tokenize
from torchtext.vocab import build_vocab_from_iterator

# Tokenize and build vocabulary
train_tokens = [word_tokenize(sentence) for sentence in sentences_train]
vocab = build_vocab_from_iterator(train_tokens, specials=["<unk>", "<pad>"])
vocab_size = len(vocab)
print ("Vocabulary size:", vocab_size)

# Set a default index for unknown tokens
unk_index = vocab["<unk>"]

# Numericalize sentences
def numericalize_sentence(sentence):
    return [vocab[token] if token in vocab else unk_index for token in word_tokenize(sentence)]

X_train = [numericalize_sentence(sentence) for sentence in sentences_train]
X_val = [numericalize_sentence(sentence) for sentence in sentences_val]
X_test = [numericalize_sentence(sentence) for sentence in sentences_test]

print (len(X_train))
print (X_train[0])

Vocabulary size: 5424
2198
[387, 285, 14, 34, 4, 6, 516, 9, 1184, 84, 62, 3, 131, 10, 14, 1506, 2]


In [57]:
print(sentences_train[6])
print(X_train[6])
print(sentences_train[3])
print(X_train[3])

One of the most boring,pointless movies I have ever seen.  
[389, 10, 3, 119, 438, 4, 1220, 161, 6, 28, 70, 192, 2]
I went on Motorola's website and followed all directions, but could not get it to pair again.
[6, 286, 24, 352, 22, 985, 5, 3910, 44, 3674, 4, 37, 87, 21, 88, 12, 9, 927, 114, 2]


In [58]:
for word in ['the', 'all', 'happy', 'sad']:
    print('{}: {}'.format(word, vocab[word]))

the: 3
all: 44
happy: 223
sad: 767


In [59]:
def pad_sequences(sequences, maxlen=None, dtype=torch.float32, padding_value=0):
    # If maxlen is not provided, determine it dynamically from the longest sequence
    if maxlen is None:
        maxlen = max(len(seq) for seq in sequences)

    # Create a tensor to store the padded sequences
    padded_sequences = torch.full((len(sequences), maxlen), padding_value, dtype=dtype)

    # Iterate over the sequences and pad them
    for i, seq in enumerate(sequences):
        end = min(len(seq), maxlen)
        padded_sequences[i, :end] = torch.tensor(seq[:end], dtype=dtype)
    
    return padded_sequences

# Pad sequences
X_train_padded = pad_sequences(X_train, maxlen=100)
X_val_padded = pad_sequences(X_val, maxlen=100)
X_test_padded = pad_sequences(X_test, maxlen=100)

print (X_train_padded[0])
X_train_padded.shape

tensor([ 387.,  285.,   14.,   34.,    4.,    6.,  516.,    9., 1184.,   84.,
          62.,    3.,  131.,   10.,   14., 1506.,    2.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.])


torch.Size([2198, 100])

## Pytorch Embedding Layer

In [60]:
class feedforward_deep_average_network(nn.Module):
    def __init__(self, word_embeddings: nn.Embedding, hidden_dim, output_dim):
        super(feedforward_deep_average_network, self).__init__()
        self.embedding = word_embeddings
        self.embedding_dim = word_embeddings.embedding_dim
        self.fc1 = nn.Linear(self.embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        pooled = torch.mean(embedded, dim=1)  # Take average over the vocab_size dimension
        out = self.fc1(pooled)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

embedding_dim = 50

# Instantiate the model
ffnn_dan_model = feedforward_deep_average_network(
    word_embeddings = nn.Embedding(vocab_size, embedding_dim),
    hidden_dim = 10, output_dim = 1)
print(ffnn_dan_model)

for name, param in ffnn_dan_model.named_parameters():
    print(f"Layer: {name}, Number of parameters: {param.numel()}")

feedforward_deep_average_network(
  (embedding): Embedding(5424, 50)
  (fc1): Linear(in_features=50, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)
Layer: embedding.weight, Number of parameters: 271200
Layer: fc1.weight, Number of parameters: 500
Layer: fc1.bias, Number of parameters: 10
Layer: fc2.weight, Number of parameters: 10
Layer: fc2.bias, Number of parameters: 1


In [61]:
def train_model(model, epochs, batch_size, train_input_tensor, train_labels_tensor, eval_input_tensor, eval_labels_tensor):

    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_dataset = TensorDataset(train_input_tensor, train_labels_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training loop
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = torch.squeeze(outputs, dim=1) # Squeeze the output tensor to match the shape of labels
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation
        val_loss, val_accuracy = evaluate_model(model, batch_size, eval_input_tensor, eval_labels_tensor)

        # Print the loss and accuracy
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.3f}, Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}')


def evaluate_model(model, batch_size, eval_input_tensor, eval_labels_tensor):
    val_dataset = TensorDataset(eval_input_tensor, eval_labels_tensor)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            outputs = torch.squeeze(outputs, dim=1)  # Squeeze the output tensor
            val_loss += criterion(outputs, labels).item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = correct / total
    return val_loss, val_accuracy

In [62]:
X_train_padded.dtype, y_train_tensor.dtype

(torch.float32, torch.float32)

In [63]:
X_train_padded, X_val_padded = X_train_padded.type(torch.LongTensor), X_val_padded.type(torch.LongTensor)  # Casting to LongTensor tensor type

train_model(
    ffnn_dan_model, epochs=10, batch_size=20,
    train_input_tensor=X_train_padded, train_labels_tensor=y_train_tensor,
    eval_input_tensor=X_val_padded, eval_labels_tensor=y_val_tensor)

Epoch 1/10, Loss: 0.693, Val Loss: 0.694, Val Accuracy: 0.480
Epoch 2/10, Loss: 0.716, Val Loss: 0.694, Val Accuracy: 0.487
Epoch 3/10, Loss: 0.697, Val Loss: 0.693, Val Accuracy: 0.491
Epoch 4/10, Loss: 0.695, Val Loss: 0.692, Val Accuracy: 0.487
Epoch 5/10, Loss: 0.682, Val Loss: 0.694, Val Accuracy: 0.487
Epoch 6/10, Loss: 0.681, Val Loss: 0.691, Val Accuracy: 0.491
Epoch 7/10, Loss: 0.723, Val Loss: 0.690, Val Accuracy: 0.491
Epoch 8/10, Loss: 0.691, Val Loss: 0.683, Val Accuracy: 0.509
Epoch 9/10, Loss: 0.633, Val Loss: 0.672, Val Accuracy: 0.589
Epoch 10/10, Loss: 0.599, Val Loss: 0.661, Val Accuracy: 0.567


In [64]:
X_test_padded = X_test_padded.type(torch.LongTensor)
test_loss, test_accuracy = evaluate_model(ffnn_dan_model, batch_size=20, eval_input_tensor=X_test_padded, eval_labels_tensor=y_test_tensor)
test_accuracy

0.6072727272727273

## Using Pretrained Word Embeddings

In [65]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 50
glove_embedding_matrix = create_embedding_matrix('glove.6B.50d.txt',
    vocab.get_stoi(), # torchtext vocab instance, get_stoi() returns the mapping from token to index
    embedding_dim)

In [66]:
glove_embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.15164   ,  0.30177   , -0.16763   , ..., -0.35652   ,
         0.016413  ,  0.10216   ],
       ...,
       [ 0.69479001, -1.15900004,  0.070302  , ..., -1.21819997,
         0.23513   ,  0.33118999],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [67]:
# We can check how many of the words we have in training are in Glove
nonzero_elements = np.count_nonzero(np.count_nonzero(glove_embedding_matrix, axis=1))
print(nonzero_elements / vocab_size)

0.7230825958702065


In [68]:
# load pre-trained embeddings into the embedding layer
golve_embedding_layer = nn.Embedding.from_pretrained(torch.tensor(glove_embedding_matrix, dtype=torch.float32), freeze=True)
print (golve_embedding_layer)

ffnn_dan_glove_model = feedforward_deep_average_network(
    word_embeddings = golve_embedding_layer,
    hidden_dim = 10, output_dim = 1)

print(ffnn_dan_glove_model)

for name, param in ffnn_dan_glove_model.named_parameters():
    print(f"Layer: {name}, Number of parameters: {param.numel()}")

Embedding(5425, 50)
feedforward_deep_average_network(
  (embedding): Embedding(5425, 50)
  (fc1): Linear(in_features=50, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)
Layer: embedding.weight, Number of parameters: 271250
Layer: fc1.weight, Number of parameters: 500
Layer: fc1.bias, Number of parameters: 10
Layer: fc2.weight, Number of parameters: 10
Layer: fc2.bias, Number of parameters: 1


In [69]:
train_model(
    ffnn_dan_glove_model, epochs=10, batch_size=20,
    train_input_tensor=X_train_padded, train_labels_tensor=y_train_tensor,
    eval_input_tensor=X_val_padded, eval_labels_tensor=y_val_tensor)

Epoch 1/10, Loss: 0.685, Val Loss: 0.697, Val Accuracy: 0.487
Epoch 2/10, Loss: 0.699, Val Loss: 0.692, Val Accuracy: 0.491
Epoch 3/10, Loss: 0.689, Val Loss: 0.686, Val Accuracy: 0.538
Epoch 4/10, Loss: 0.692, Val Loss: 0.681, Val Accuracy: 0.604
Epoch 5/10, Loss: 0.664, Val Loss: 0.675, Val Accuracy: 0.625
Epoch 6/10, Loss: 0.644, Val Loss: 0.667, Val Accuracy: 0.636
Epoch 7/10, Loss: 0.649, Val Loss: 0.661, Val Accuracy: 0.640
Epoch 8/10, Loss: 0.693, Val Loss: 0.653, Val Accuracy: 0.636
Epoch 9/10, Loss: 0.596, Val Loss: 0.649, Val Accuracy: 0.622
Epoch 10/10, Loss: 0.588, Val Loss: 0.643, Val Accuracy: 0.658


In [70]:
test_loss, test_accuracy = evaluate_model(ffnn_dan_glove_model, batch_size=20, eval_input_tensor=X_test_padded, eval_labels_tensor=y_test_tensor)
test_accuracy

0.7454545454545455

## LSTMs with pytorch

Recall that we were "averaging" the word embeddings in order to get a sentence representation... but can we do better? In class, we discussed the use of RNNs (e.g., LSTM) as a way of "summarizing" sentence meaning in its hidden vectors. As it turns out, it is very easy to use an LSTM instead of doing an average pooling; you only need to specify an output dimension for the LSTM, i.e., the size of your "sentence summary vector":

In [71]:
class feedforward_deep_average_lstm_network(nn.Module):
    def __init__(self, word_embeddings: nn.Embedding, dropout, lstm_dim, hidden_dim, output_dim):
        super(feedforward_deep_average_lstm_network, self).__init__()
        self.embedding = word_embeddings
        self.embedding_dim = word_embeddings.embedding_dim
        self.lstm = nn.LSTM(self.embedding_dim, lstm_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(lstm_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        # [:] in the first dimension means selecting all batches.
        # [-1] in the second dimension means selecting the last time step.
        # [:] in the third dimension means selecting all features.
        lstm_out = self.dropout(lstm_out[:, -1, :])  # Using the last output only
        out = torch.relu(self.fc1(lstm_out))
        out = self.sigmoid(self.fc2(out))
        return out

In [72]:
ffnn_dan_glove_lstm_model = feedforward_deep_average_lstm_network(
    word_embeddings = golve_embedding_layer,
    dropout = 0.5,
    lstm_dim = 16, hidden_dim = 10, output_dim = 1)

print(ffnn_dan_glove_lstm_model)

for name, param in ffnn_dan_glove_lstm_model.named_parameters():
    print(f"Layer: {name}, Number of parameters: {param.numel()}")

feedforward_deep_average_lstm_network(
  (embedding): Embedding(5425, 50)
  (lstm): LSTM(50, 16)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=16, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Layer: embedding.weight, Number of parameters: 271250
Layer: lstm.weight_ih_l0, Number of parameters: 3200
Layer: lstm.weight_hh_l0, Number of parameters: 1024
Layer: lstm.bias_ih_l0, Number of parameters: 64
Layer: lstm.bias_hh_l0, Number of parameters: 64
Layer: fc1.weight, Number of parameters: 160
Layer: fc1.bias, Number of parameters: 10
Layer: fc2.weight, Number of parameters: 10
Layer: fc2.bias, Number of parameters: 1


In [73]:
train_model(
    ffnn_dan_glove_lstm_model, epochs=10, batch_size=20,
    train_input_tensor=X_train_padded, train_labels_tensor=y_train_tensor,
    eval_input_tensor=X_val_padded, eval_labels_tensor=y_val_tensor)

Epoch 1/10, Loss: 0.693, Val Loss: 0.693, Val Accuracy: 0.513
Epoch 2/10, Loss: 0.692, Val Loss: 0.693, Val Accuracy: 0.498
Epoch 3/10, Loss: 0.693, Val Loss: 0.693, Val Accuracy: 0.505
Epoch 4/10, Loss: 0.694, Val Loss: 0.693, Val Accuracy: 0.505
Epoch 5/10, Loss: 0.694, Val Loss: 0.693, Val Accuracy: 0.513
Epoch 6/10, Loss: 0.695, Val Loss: 0.693, Val Accuracy: 0.513
Epoch 7/10, Loss: 0.694, Val Loss: 0.693, Val Accuracy: 0.505
Epoch 8/10, Loss: 0.694, Val Loss: 0.693, Val Accuracy: 0.487
Epoch 9/10, Loss: 0.695, Val Loss: 0.693, Val Accuracy: 0.487
Epoch 10/10, Loss: 0.698, Val Loss: 0.693, Val Accuracy: 0.487


In [74]:
test_loss, test_accuracy = evaluate_model(ffnn_dan_glove_lstm_model, batch_size=20, eval_input_tensor=X_test_padded, eval_labels_tensor=y_test_tensor)
test_accuracy

0.4618181818181818